In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline
from sklearn import set_config; set_config(display = "diagram")
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('final_le_wagon_project_data.csv')

In [3]:
df.head(5)

,sr_no,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,...,shortlist_uni_2,admit_uni_2,shortlist_uni_3,admit_uni_3,shortlist_uni_4,admit_uni_4,shortlist_uni_5,admit_uni_5,shortlist_uni_6,admit_uni_6
0,1,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,...,0,0,0,0,0,0,1,0,0,0
2,3,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(['sr_no', 'admit_uni_1', 'admit_uni_2', 'admit_uni_3', 'admit_uni_4', 'admit_uni_5', 'admit_uni_6'], axis=1)
df = df.dropna().copy()

In [5]:
df.head(10)

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,shortlist_uni_2,shortlist_uni_3,shortlist_uni_4,shortlist_uni_5,shortlist_uni_6
0,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,0,0,0,0,0
1,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,0,0,0,1,0
2,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,0,0,0,0,0
3,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,0,0,0,0,0
4,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,0,0,0,0,0
5,Male,General,87.4,92.6,79.9,Engineer,99.63,21,0,0,1,0,0,0
6,Male,NC_OBC,99.3,89.7,73.6,Engineer,95.01,25,0,1,0,1,1,1
7,Male,General,95.0,86.8,71.1,Engineer,99.26,26,0,0,0,0,0,0
8,Male,General,87.4,79.6,70.1,Engineer,99.22,28,0,0,0,0,0,1
9,Female,NC_OBC,85.5,84.3,70.9,Non-Engineer,91.50,14,0,0,0,0,1,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3940 entries, 0 to 3939
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                3940 non-null   object 
 1   category              3940 non-null   object 
 2   grade_10              3940 non-null   float64
 3   grade_12              3940 non-null   float64
 4   graduation_score      3940 non-null   float64
 5   graduation_type       3940 non-null   object 
 6   admission_test_score  3940 non-null   float64
 7   work_exp_months       3940 non-null   int64  
 8   shortlist_uni_1       3940 non-null   int64  
 9   shortlist_uni_2       3940 non-null   int64  
 10  shortlist_uni_3       3940 non-null   int64  
 11  shortlist_uni_4       3940 non-null   int64  
 12  shortlist_uni_5       3940 non-null   int64  
 13  shortlist_uni_6       3940 non-null   int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 431.1+ KB


In [7]:
final_preprocessor = ColumnTransformer([
    ('num_encoder', RobustScaler(), make_column_selector(dtype_exclude="object")),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', drop = 'first', sparse=False), make_column_selector(dtype_include="object"))])

In [8]:
final_preprocessor

ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1257c6f50>),
                                ('cat_encoder',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1257c7460>)])

In [9]:
X = df.drop(columns=["shortlist_uni_1", "shortlist_uni_2", "shortlist_uni_3",
                    "shortlist_uni_4", "shortlist_uni_5", "shortlist_uni_6" ])

y1 = df["shortlist_uni_1"]
y2 = df["shortlist_uni_2"]
y3 = df["shortlist_uni_3"]
y4 = df["shortlist_uni_4"]
y5 = df["shortlist_uni_5"]
y6 = df["shortlist_uni_6"]

In [10]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y3, test_size=0.3, random_state=42)

# Print the shapes of the resulting arrays to verify the split
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2758, 8)
y_train shape: (2758,)
X_test shape: (1182, 8)
y_test shape: (1182,)


In [11]:
from lazypredict.Supervised import LazyClassifier

In [12]:
# Create an instance of the LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Train the LazyClassifier on the training data
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Print the model performance summary
print(models)

100%|███████████████████████████████████████████| 29/29 [00:04<00:00,  7.13it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.92               0.78     0.78      0.92   
BaggingClassifier                  0.92               0.76     0.76      0.91   
LGBMClassifier                     0.92               0.76     0.76      0.92   
DecisionTreeClassifier             0.90               0.75     0.75      0.90   
RandomForestClassifier             0.92               0.74     0.74      0.91   
AdaBoostClassifier                 0.92               0.72     0.72      0.91   
NearestCentroid                    0.59               0.69     0.69      0.67   
ExtraTreeClassifier                0.86               0.63     0.63      0.85   
ExtraTreesClassifier               0.90               0.63     0.63      0.88   
GaussianNB                         0.87               0.60     0.60      0.86   
QuadraticDiscriminantAnalysi

In [ ]:
df.head()

In [ ]:
pd.DataFrame(final_preprocessor.fit_transform(X_train)).head()

In [ ]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe1 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', KNeighborsClassifier(n_neighbors=3))
])

pipe1

In [ ]:
pipe1.fit(X_train, y_train)

In [ ]:
pipe1['preprocessing'].transformers_

In [ ]:
col_num = pipe1['preprocessing'].transformers_[0][2]
col_num

In [ ]:
pipe1['preprocessing'].transformers_[1][1].get_feature_names_out()

In [ ]:
col_cat = list(pipe1['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

In [ ]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [ ]:
len(col_preproc)

In [ ]:
pd.DataFrame(data=pipe1['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

In [ ]:
X_train.info()

In [ ]:
scores = cross_val_score(pipe1, X_train, y_train, cv=5, scoring="accuracy")
score = scores.mean()

print(scores)
print(scores.mean())

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Create a list of potential values for k
k_values = list(range(1, 11))

# Create an empty list to store the cross-validation scores
cv_scores = []

# Loop over each value of k
for k in k_values:
    # Create a KNN classifier with k neighbors
    knn = KNeighborsClassifier(n_neighbors=k)
    # Compute the cross-validation score for this KNN classifier
    scores = cross_val_score(pipe1, X_train, y_train, cv=5)
    # Compute the mean cross-validation score
    mean_score = scores.mean()
    # Append the mean score to the list of cross-validation scores
    cv_scores.append(mean_score)

# Find the optimal value of k
optimal_k = k_values[cv_scores.index(max(cv_scores))]

# Print the optimal value of k and the corresponding accuracy
print("Optimal k:", optimal_k)
print("Accuracy:", max(cv_scores))


In [ ]:
# Train the model using the training data
pipe1.fit(X_train, y_train)

# Use the model to make predictions on the testing data
y_pred = pipe1.predict(X_test)

# Evaluate the accuracy of the model
accuracy = pipe1.score(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
#import pickle

# Export pipeline as pickle file
#with open("pipeline1.pkl", "wb") as file:
#    pickle.dump(pipe1, file)

In [ ]:
# Load pipeline from pickle file
#my_pipeline1 = pickle.load(open("pipeline1.pkl", "rb"))

#my_pipeline1

In [ ]:
#new_data = pd.read_csv("/Users/prasadsawant/Downloads/new_data.csv")

In [ ]:
#new_data

In [ ]:
#predicted_class = my_pipeline1.predict(new_data)
#predicted_class

In [ ]:
pip install tensorflow

In [ ]:
# Import the necessary libraries
import tensorflow as tf

In [ ]:
#X_train = np.asarray(X_train).astype(np.float32)

In [ ]:
X_train

In [ ]:
pd.DataFrame(data=pipe1['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

In [ ]:
X_train.info()

In [ ]:
# Create the model
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100)

# Plot the loss over time
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Plot the predicted values against the actual values
plt.scatter(X_test, y_test, label='Actual')
plt.scatter(X_test, predictions, label='Predicted')
plt.title('Predictions vs Actual Values')
plt.legend()
plt.show()
